In [1]:
import os
import torch
import torchvision

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import ModelCheckpoint

from model import PlaceholderModel
from pathlib import Path
from argparse import Namespace

import tqdm

In [2]:
checkpoint_dir = "/scratch/ns4486/capstone/checkpoints/conv_ae/arch_2/AE_batch1/NUM-5/checkpoints/last.ckpt"

In [3]:
data_path = '/scratch/ns4486/numerical-relativity-interpolation/Proca_fiducial_scaled_cropped.hdf5'
model = PlaceholderModel.load_from_checkpoint(checkpoint_dir, hparams = {}, data_path=data_path)

model = model.eval()

In [4]:
device = torch.device('cuda:0')

In [5]:
model = model.to(device)

In [5]:
# from torchsummary import summary

# summary(model.decoder, (32, 9, 9, 9), device='cpu')

In [6]:
from loader import SingleChannelDataset

In [7]:
dataset = SingleChannelDataset(data_path)

In [8]:
import numpy as np

In [15]:
outputs = None
inputs = None

with torch.no_grad():
    for i in tqdm.tqdm(range(len(dataset))):
    #     print(dataset[i][2].shape)
    #     print(torch.cat((dataset[i][0], dataset[i][1])).shape)
#         print(dataset[i][0].shape)
    #     break
        x1 = dataset[i][0].unsqueeze(0)
        x2 = dataset[i][1].unsqueeze(0)
        x1 = x1.to(device)
        x2 = x2.to(device)
        z = model(x1, x2)
        y_hat = model.decoder(z)
    #     print(y_hat.squeeze(0).squeeze(0).shape)

        if inputs is None:
            inputs = torch.cat((dataset[i][0], dataset[i][1]))
#             print(inputs.shape)
            inputs = inputs.unsqueeze(0)
        else:
            t = torch.cat((dataset[i][0], dataset[i][1])).unsqueeze(0)
#             print(t.shape)
            inputs = torch.cat((inputs, t))

        if outputs is None:
            outputs = y_hat.cpu().squeeze(0).detach().numpy()
        else:
            outputs = np.concatenate((outputs, y_hat.cpu().squeeze(0).detach().numpy()), axis=0)

100%|██████████| 407/407 [04:33<00:00,  1.49it/s]


In [16]:
outputs = outputs.reshape((407, 1, 72, 72, 72))

In [17]:
outputs.shape

(407, 1, 72, 72, 72)

In [20]:
inputs = np.array(inputs)
inputs.shape

(407, 2, 72, 72, 72)

In [21]:
with open('results_num_19.npy', 'wb') as f:
    np.save(f, inputs)
    np.save(f, outputs)

In [14]:
# with open('results_num_19.npy', 'rb') as f:
#     inputs = np.load(f)
#     outputs = np.load(f)

In [5]:
import numpy as np
import cv2
import os
from PIL import Image

def transform_tensor_to_video_frames(frame_tensor):
    # shape of frame_tensor is (num_frames, spatial_x, spatial_y)

    num_frames = frame_tensor.shape[0]

    b_marginal = np.array(frame_tensor)
    b_norm = np.uint8(255 * (b_marginal - b_marginal.min()) / (b_marginal.mean() - b_marginal.min()))

    all_array = []

    for i in range(num_frames):
        all_array.append(np.array(Image.fromarray(b_norm[i], 'L').convert('RGB')))

    return all_array



def save_video(frame_list, save_path, video_shape):
    out = cv2.VideoWriter(
        save_path,
        cv2.VideoWriter_fourcc(*'XVID'), 
        10, 
        video_shape
    )

    for i in range(len(frame_list)):
        out.write(frame_list[i])
    out.release()

    return



def generate_video_from_tensor(frame_tensor, save_path):
    # shape of frame_tensor is (num_frames, spatial_x, spatial_y)

    all_array = transform_tensor_to_video_frames(frame_tensor)
    save_video(all_array, save_path, (72, 72))

    return

'''
video_path = '/Users/nikhilvs/repos/nyu/numerical-relativity-interpolation/notebooks/test_new_2.mp4'

generate_video_from_tensor(
    a[:, 0, :, :, :],
    video_path
)


from IPython.display import Video

Video(
    video_path, 
    embed = True,
    width = 500,
    height = 500
)
'''


def generate_input_target_predicted_video(input_tensor, target, predicted, save_path):
    # input shape is (num_frames, 2, spatial_x, spatial_y)
    # target shape is (num_frames, 1, spatial_x, spatial_y)

    # stutter video is even frames from input shape
    # target video is all frames from target
    # predicted video is all frames from predicted

    input_even = input_tensor[::2, 0, :, :, 36]
    input_repeated = np.repeat(input_even, [2], axis = 0)

    input_video_frames = transform_tensor_to_video_frames(input_repeated)
    target_video_frames = transform_tensor_to_video_frames(target[:, 0, :, 36])
    predicted_video_frames = transform_tensor_to_video_frames(predicted[:, 0, :, 36])

    all_video_frames = []

    for i in range(len(input_video_np.shape)):
        input_frame = input_video_frames[i]
        target_frame = target_video_frames[i]
        predicted_frame = predicted_video_frames[i]

        video_frame = np.concatenate((input_frame, target_frame, predicted_frame), axis = 1)
        all_video_frames.append(video_frame)
    
    save_video(all_video_frames, save_path, (72, 72 * 3))

    return


def generate_input_target_video(input_tensor, target, save_path):
    # input shape is (num_frames, 2, spatial_x, spatial_y)
    # target shape is (num_frames, 1, spatial_x, spatial_y)

    # stutter video is even frames from input shape
    # target video is all frames from target
    # predicted video is all frames from predicted

    input_even = input_tensor[::2, 0, :, :, 36]
    input_repeated = np.repeat(input_even, [2], axis = 0)

    input_video_frames = transform_tensor_to_video_frames(input_repeated)
    target_video_frames = transform_tensor_to_video_frames(target[:, 0, :, :, 36])

    all_video_frames = []
    
    num_frames = min(len(input_video_frames), len(target_video_frames))

    for i in range(num_frames):
        input_frame = input_video_frames[i]
        target_frame = target_video_frames[i]

        video_frame = np.concatenate((input_frame, target_frame), axis = 1)
        all_video_frames.append(video_frame)
    
    save_video(all_video_frames, save_path, (72, 72 * 2))

    return


def generate_input_target_video_1(input_tensor, target, save_path):
    # input shape is (num_frames, 2, spatial_x, spatial_y)
    # target shape is (num_frames, 1, spatial_x, spatial_y)

    # stutter video is even frames from input shape
    # target video is all frames from target
    # predicted video is all frames from predicted
    
    input_tensor = np.array(input_tensor)
    target = np.array(target[:, 0, 71, :, :])

    input_even = input_tensor[::2, 0, 71, :, :]
    input_repeated = np.repeat(input_even, [2], axis = 0)

    
    num_frames = min(input_repeated.shape[0], target.shape[0])

    print(
        (input_repeated[:num_frames, :, :].shape, target[:num_frames, :, :].shape)
    )
    
    video_np = np.concatenate(
        (input_repeated[:num_frames, :, :], target[:num_frames, :, :]), 
        axis = 1
    )
    
    all_video_frames = transform_tensor_to_video_frames(video_np)
    save_video(all_video_frames, save_path, (72, 72*2))

    return

In [7]:
inputs.shape

(407, 2, 72, 72, 72)

In [22]:
generate_input_target_video_1(
    inputs[1:], 
    outputs,
    'test_new_4.mp4'
)

((406, 72, 72), (406, 72, 72))


In [40]:
from IPython.display import Video

Video(
    'test_new_3.mp4', 
    embed = True,
    width = 500,
    height = 500
)

In [21]:
from PIL import Image

all_array = []

for i in range(407):
    img_arr = outputs[i, 0]
    img_arr = (img_arr) / (14)
#     img_arr = img_arr.astype(int)
    all_array.append(np.array(Image.fromarray(outputs[i, 0], 'L').convert('RGB')))
#     all_array.append(np.array(Image.fromarray(b_norm[i], 'L').convert('RGB')))

In [22]:
import cv2

out = cv2.VideoWriter(
    'all_nn_outputs.avi', 
    cv2.VideoWriter_fourcc(*'DIVX'), 
    10, 
    (72, 72)
)

for i in range(len(all_array)):
    out.write(all_array[i])
out.release()